##Read CSV files into Dataframe

In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = (SparkSession.builder.appName("dataPrep").getOrCreate())

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [0]:
schema = StructType([
  StructField("Order ID", StringType(), True),
  StructField("Product", StringType(), True),
  StructField("Quantity Order", StringType(), True),
  StructField("Price Each", StringType(), True),
  StructField("Order Date", StringType(), True),
  StructField("Purchase Address", StringType(), True)
])

In [0]:
sales_data_fpath = "dbfs:/FileStore/salesdata/raw"
sales_raw_df = (spark.read.format("csv")
                 .option("header", True)
                 .schema(schema)
                 .load(sales_data_fpath))

In [0]:
sales_raw_df.show(5)

+--------+--------------------+--------------+----------+--------------+--------------------+
Order ID| Product|Quantity Order|Price Each| Order Date| Purchase Address|
+--------+--------------------+--------------+----------+--------------+--------------------+
 295665| Macbook Pro Laptop| 1| 1700|12/30/19 00:01|136 Church St, Ne...|
 295666| LG Washing Machine| 1| 600.0|12/29/19 07:03|562 2nd St, New Y...|
 295667|USB-C Charging Cable| 1| 11.95|12/12/19 18:21|277 Main St, New ...|
 295668| 27in FHD Monitor| 1| 149.99|12/22/19 15:13|410 6th St, San F...|
 295669|USB-C Charging Cable| 1| 11.95|12/18/19 12:38|43 Hill St, Atlan...|
+--------+--------------------+--------------+----------+--------------+--------------------+
only showing top 5 rows

In [0]:
sales_raw_df.printSchema()

root
-- Order ID: string (nullable = true)
-- Product: string (nullable = true)
-- Quantity Order: string (nullable = true)
-- Price Each: string (nullable = true)
-- Order Date: string (nullable = true)
-- Purchase Address: string (nullable = true)

In [0]:
spark.sql("CREATE DATABASE IF NOT EXISTS sales_db")

Out[8]: DataFrame[]

In [0]:
spark.sql("use sales_db")

Out[9]: DataFrame[]

In [0]:
# spark.sql("drop table sales_raw")
spark.sql(""" create table if not exists sales_raw(
      OrderId string,
      Product string,
      QuantityOrdered string,
      PriceEach string,
      OrderDate string,
      PurchaseAddress string
)""")

Out[10]: DataFrame[]

##Insert dataFrame data to Sales raw table

In [0]:
sales_raw_df.createOrReplaceTempView("temp_sales_raw")

In [0]:
spark.sql("select * from temp_sales_raw").show(5, False)

+--------+--------------------+--------------+----------+--------------+--------------------------------------+
Order ID|Product |Quantity Order|Price Each|Order Date |Purchase Address |
+--------+--------------------+--------------+----------+--------------+--------------------------------------+
295665 |Macbook Pro Laptop |1 |1700 |12/30/19 00:01|136 Church St, New York City, NY 10001|
295666 |LG Washing Machine |1 |600.0 |12/29/19 07:03|562 2nd St, New York City, NY 10001 |
295667 |USB-C Charging Cable|1 |11.95 |12/12/19 18:21|277 Main St, New York City, NY 10001 |
295668 |27in FHD Monitor |1 |149.99 |12/22/19 15:13|410 6th St, San Francisco, CA 94016 |
295669 |USB-C Charging Cable|1 |11.95 |12/18/19 12:38|43 Hill St, Atlanta, GA 30301 |
+--------+--------------------+--------------+----------+--------------+--------------------------------------+
only showing top 5 rows

In [0]:
spark.sql("describe temp_sales_raw").show()

+----------------+---------+-------+
 col_name|data_type|comment|
+----------------+---------+-------+
 Order ID| string| null|
 Product| string| null|
 Quantity Order| string| null|
 Price Each| string| null|
 Order Date| string| null|
Purchase Address| string| null|
+----------------+---------+-------+

In [0]:
spark.sql("describe sales_db.sales_raw").show()

+---------------+---------+-------+
 col_name|data_type|comment|
+---------------+---------+-------+
 OrderId| string| null|
 Product| string| null|
QuantityOrdered| string| null|
 PriceEach| string| null|
 OrderDate| string| null|
PurchaseAddress| string| null|
+---------------+---------+-------+

In [0]:
spark.sql("""INSERT overwrite table sales_db.sales_raw
      SELECT `Order ID`, `Product`, `Quantity Order`, `Price Each`, `Order Date`, `Purchase Address`
      FROM temp_sales_raw""")

Out[15]: DataFrame[]

In [0]:
spark.sql("SELECT * FROM sales_db.sales_raw").show(5)

+-------+--------------------+---------------+---------+--------------+--------------------+
OrderId| Product|QuantityOrdered|PriceEach| OrderDate| PurchaseAddress|
+-------+--------------------+---------------+---------+--------------+--------------------+
 295665| Macbook Pro Laptop| 1| 1700|12/30/19 00:01|136 Church St, Ne...|
 295666| LG Washing Machine| 1| 600.0|12/29/19 07:03|562 2nd St, New Y...|
 295667|USB-C Charging Cable| 1| 11.95|12/12/19 18:21|277 Main St, New ...|
 295668| 27in FHD Monitor| 1| 149.99|12/22/19 15:13|410 6th St, San F...|
 295669|USB-C Charging Cable| 1| 11.95|12/18/19 12:38|43 Hill St, Atlan...|
+-------+--------------------+---------------+---------+--------------+--------------------+
only showing top 5 rows

In [0]:
temp_sales_df = spark.sql("""select cast(OrderID as int) as OrderId,
                                Product,
                                QuantityOrdered,
                                PriceEach,
                                to_timestamp(OrderDate, 'MM/dd/yy HH:mm') as OrderDate,
                                PurchaseAddress,
                                split(purchaseaddress, ",")[1] as city,
                                split(split(purchaseaddress, ",")[2], " ")[1] as state,
                                year(to_timestamp(OrderDate, 'MM/dd/yy HH:mm')) as ReportYear,
                                month(to_timestamp(OrderDate, 'MM/dd/yy HH:mm')) as Month
                             from sales_raw
                             where orderid is not null
                               and orderid != "Order ID"

""")

In [0]:
temp_sales_df.show(5)

+-------+--------------------+---------------+---------+-------------------+--------------------+--------------+-----+----------+-----+
OrderId| Product|QuantityOrdered|PriceEach| OrderDate| PurchaseAddress| city|state|ReportYear|Month|
+-------+--------------------+---------------+---------+-------------------+--------------------+--------------+-----+----------+-----+
 295665| Macbook Pro Laptop| 1| 1700|2019-12-30 00:01:00|136 Church St, Ne...| New York City| NY| 2019| 12|
 295666| LG Washing Machine| 1| 600.0|2019-12-29 07:03:00|562 2nd St, New Y...| New York City| NY| 2019| 12|
 295667|USB-C Charging Cable| 1| 11.95|2019-12-12 18:21:00|277 Main St, New ...| New York City| NY| 2019| 12|
 295668| 27in FHD Monitor| 1| 149.99|2019-12-22 15:13:00|410 6th St, San F...| San Francisco| CA| 2019| 12|
 295669|USB-C Charging Cable| 1| 11.95|2019-12-18 12:38:00|43 Hill St, Atlan...| Atlanta| GA| 2019| 12|
+-------+--------------------+---------------+---------+-------------------+--------------------+--------------+-----+----------+-----+
only showing top 5 rows

In [0]:
temp_sales_df.createOrReplaceTempView("temp_sales")

In [0]:
spark.sql("describe temp_sales").show()

+---------------+---------+-------+
 col_name|data_type|comment|
+---------------+---------+-------+
 OrderId| int| null|
 Product| string| null|
QuantityOrdered| string| null|
 PriceEach| string| null|
 OrderDate|timestamp| null|
PurchaseAddress| string| null|
 city| string| null|
 state| string| null|
 ReportYear| int| null|
 Month| int| null|
+---------------+---------+-------+

In [0]:
spark.sql("""
      create table if not exists sales(
      OrderID int,
      Product string,
      QuantityOrdered int,
      PriceEach int,
      OrderDate timestamp,
      PurchaseAddress string,
      city string,
      state string,
      ReportYear int,
      Month int
    )
    using Parquet
    partitioned by (ReportYear, Month)
    options('compression' = 'snappy')
    location 'dbfs:/FileStore/salesdata/published'
    """)

Out[21]: DataFrame[]

In [0]:
#spark.sql("drop table sales")
spark.sql("describe sales").show()

+--------------------+---------+-------+
 col_name|data_type|comment|
+--------------------+---------+-------+
 OrderID| int| null|
 Product| string| null|
 QuantityOrdered| int| null|
 PriceEach| int| null|
 OrderDate|timestamp| null|
 PurchaseAddress| string| null|
 city| string| null|
 state| string| null|
 ReportYear| int| null|
 Month| int| null|
# Partition Infor...| | |
 # col_name|data_type|comment|
 ReportYear| int| null|
 Month| int| null|
+--------------------+---------+-------+

In [0]:
spark.sql("""
    insert into sales
    select
      OrderID,
      Product,
      cast(QuantityOrdered as int),
      cast(PriceEach as float),
      OrderDate,
      PurchaseAddress,
      city,
      state,
      ReportYear,
      Month
    from temp_sales
""")

Out[23]: DataFrame[]

In [0]:
%sql
select * from sales limit 5

OrderID,Product,QuantityOrdered,PriceEach,OrderDate,PurchaseAddress,city,state,ReportYear,Month
141234,iPhone,1,700,2019-01-22T21:25:00.000+0000,"944 Walnut St, Boston, MA 02215",Boston,MA,2019,1
141235,Lightning Charging Cable,1,14,2019-01-28T14:15:00.000+0000,"185 Maple St, Portland, OR 97035",Portland,OR,2019,1
141236,Wired Headphones,2,11,2019-01-17T13:33:00.000+0000,"538 Adams St, San Francisco, CA 94016",San Francisco,CA,2019,1
141237,27in FHD Monitor,1,149,2019-01-05T20:33:00.000+0000,"738 10th St, Los Angeles, CA 90001",Los Angeles,CA,2019,1
141238,Wired Headphones,1,11,2019-01-25T11:59:00.000+0000,"387 10th St, Austin, TX 73301",Austin,TX,2019,1
